In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from typing import Callable, List
from glasses.utils.Storage import ForwardModuleStorage
from torch import nn
from glasses.models.base import Encoder
import torch

class Encoder(nn.Module):
    """Base encoder class, it allows to access the inner features.
    """
    def __init__(self, stages: Callable[[Encoder], List[nn.Module]] = lambda enc: enc.layers[:-1], 
                 features_widths : Callable[[Encoder], List[int]] = lambda enc: enc.widths[:-1]):
        super().__init__()
        self.storage = None
        self.stages = stages
        self.features_widths = features_widths
        
    @property
    def features(self):
        if self.storage is None:
            self.storage = ForwardModuleStorage(self, [*self.stages(self)])
        return list(self.storage.state.values())

In [51]:
# class ResNetStagesExtractor()

class DummyEncoder(Encoder):
    def __init__(self, start_features=16, widths=[32, 64, 128, 256, 512], stages=None, **kwargs):
        super().__init__(stages=stages)
        in_out_widths = zip([start_features, *widths], widths)
        self.layers = nn.ModuleList([nn.Conv2d(in_f, out_f, **kwargs) for in_f, out_f in in_out_widths])
      
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

x = torch.randn(1, 16, 56, 56)
enc = DummyEncoder( kernel_size=3)
# enc = DummyEncoder(stages = lambda enc: [enc.layers[1]], kernel_size=3)
print(enc)
enc.features
enc(x).shape
[print(f.shape) for f in enc.features]

DummyEncoder(
  (layers): ModuleList(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  )
)


TypeError: 'NoneType' object is not callable

In [1]:
from glasses.models.segmentation.fpn import *
from glasses.models.segmentation.unet import *
from functools import partial
from glasses.models import *
from glasses.models.classification.resnet import ResNetBottleneckBlock

model = PFPN(encoder=partial(ResNetEncoder, block=ResNetBottleneckBlock, depths=[2,2,2,2]))
# model = FPN()
# model = PPFN()
out = model(torch.randn(1,1,224,224))
print(out.shape)
# model

# [o.shape for o in out]
# model.encoder.features_widths

torch.Size([1, 2, 224, 224])


# AutoModel

In [2]:
from glasses.models.AutoModel import AutoModel
from glasses.models.AutoConfig import AutoConfig
import torch

In [3]:

unet = UNet.from_encoder(partial(AutoModel.from_name, 'resnet18'))
model.summary(input_shape=(1,384,384))

NameError: name 'UNet' is not defined

# Benchmark

In [3]:
a = [1,2,3]
b = [1,2,3]

a == b

True

In [10]:
 {'regnetx_002' : RegNetScaler()(w_0 = 24, w_a = 36.44, w_m = 2.49, group_w = 8, depth = 13 ),
    'regnetx_004' : RegNetScaler()(w_0 = 24, w_a = 24.48, w_m = 2.54, group_w = 16, depth = 22 ),
    'regnetx_006' : RegNetScaler()(w_0 = 48, w_a = 36.97, w_m = 2.24, group_w = 24, depth = 16 ),
    'regnetx_008' : RegNetScaler()(w_0 = 56, w_a = 35.73, w_m = 2.28, group_w = 16, depth = 16 ),
    'regnetx_016' : RegNetScaler()(w_0 = 80, w_a = 34.01, w_m = 2.25, group_w = 24, depth = 18 ),
    'regnetx_032' : RegNetScaler()(w_0 = 88, w_a = 26.31, w_m = 2.25, group_w = 48, depth = 25 ),
    'regnetx_040' : RegNetScaler()(w_0 = 96, w_a = 38.65, w_m = 2.43, group_w = 40, depth = 23 ),
    'regnetx_064' : RegNetScaler()(w_0 = 184, w_a = 60.83, w_m = 2.07, group_w = 56, depth = 17 ),
    'regnetx_080' : RegNetScaler()(w_0 = 80, w_a = 49.56, w_m = 2.88, group_w = 120, depth = 23 ),
    'regnetx_120' : RegNetScaler()(w_0 = 168, w_a = 73.36, w_m = 2.37, group_w = 112, depth = 19 ),
    'regnetx_160' : RegNetScaler()(w_0 = 216, w_a = 55.59, w_m = 2.1, group_w = 128, depth = 22 ),
    'regnetx_320' : RegNetScaler()(w_0 = 320, w_a = 69.86, w_m = 2.0, group_w = 168, depth = 23 ),
    'regnety_002' : RegNetScaler()(w_0 = 24, w_a = 36.44, w_m = 2.49, group_w = 8, depth = 13 ),
    'regnety_004' : RegNetScaler()(w_0 = 48, w_a = 27.89, w_m = 2.09, group_w = 8, depth = 16 ),
    'regnety_006' : RegNetScaler()(w_0 = 48, w_a = 32.54, w_m = 2.32, group_w = 16, depth = 15 ),
    'regnety_008' : RegNetScaler()(w_0 = 56, w_a = 38.84, w_m = 2.4, group_w = 16, depth = 14 ),
    'regnety_016' : RegNetScaler()(w_0 = 48, w_a = 20.71, w_m = 2.65, group_w = 24, depth = 27 ),
    'regnety_032' : RegNetScaler()(w_0 = 80, w_a = 42.63, w_m = 2.66, group_w = 24, depth = 21 ),
    'regnety_040' : RegNetScaler()(w_0 = 96, w_a = 31.41, w_m = 2.24, group_w = 64, depth = 22 ),
    'regnety_064' : RegNetScaler()(w_0 = 112, w_a = 33.22, w_m = 2.27, group_w = 72, depth = 25 ),
    'regnety_080' : RegNetScaler()(w_0 = 192, w_a = 76.82, w_m = 2.19, group_w = 56, depth = 17 ),
    'regnety_120' : RegNetScaler()(w_0 = 168, w_a = 73.36, w_m = 2.37, group_w = 112, depth = 19 ),
    'regnety_160' : RegNetScaler()(w_0 = 200, w_a = 106.23, w_m = 2.48, group_w = 112, depth = 18 ),
    'regnety_320' : RegNetScaler()(w_0 = 232, w_a = 115.89, w_m = 2.53, group_w = 232, depth = 20)}

# Clone

In [26]:
from glasses.models import AutoModel, AutoConfig, EfficientNetLite
import timm
from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
# dst = AutoModel.from_name('resnet34d')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [27]:
dst = clone_model(src, dst).eval()

In [15]:
timm.list_models()

['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn107',
 'dpn131',
 'eca_vovnet39b',
 'ecaresnet18',
 'ecaresnet50',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnetlight',
 'ecaresnext26tn_32x4d',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',


**SE features are in_features / reduction not out_features / reduction**

In [4]:
dst = clone_model(src, dst).eval()

In [29]:
import torch

transform = AutoConfig.from_name('efficientnet_b0').transform

benchmark(src.cuda(), transform, batch_size=128)

Using downloaded and verified file: ./imagenet_val_targets.pkl


(0.69948, 0.89662, 76.25272679328918)

In [19]:

transform = AutoConfig.from_name('resnet18').transform

benchmark(src.cuda(), transform, batch_size=128)

Using downloaded and verified file: ./imagenet_val_targets.pkl


KeyboardInterrupt: 

# Pretrained weights

In [7]:
from pathlib import Path

root = Path('/home/zuppif/Documents/glasses-weights/')

models = map(lambda x: x.stem, root.glob('*.pth'))

for model in models:
    print(f"'{model}': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/{model}.pth?raw=true'),")

'densenet201': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/densenet201.pth?raw=true'),
'resnet26d': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/resnet26d.pth?raw=true'),
'resnet18': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/resnet18.pth?raw=true'),
'regnety_032': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/regnety_032.pth?raw=true'),
'densenet169': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/densenet169.pth?raw=true'),
'regnetx_004': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/regnetx_004.pth?raw=true'),
'resnet34d': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/resnet34d.pth?raw=true'),
'regnetx_016': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weig

## FPN